# Parameter initialization using Gibbs Sampling

In [62]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm, invgamma, norm
import pickle
import time
import math

In [63]:
%load_ext autoreload
%autoreload 2
from elec_forecast.gibbs_sampling_model import GibbsSamplingModel
from elec_forecast.rm_estimation import RmEstimation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Required data loading

### Electricity data

In [64]:
mypath="data/"
df=pd.read_csv(mypath + "cdc_conso_daytypes.csv",sep=",",encoding='latin-1')
df.index=range(0,len(df))

In [65]:
df=df.sort_values(by=['Date'])
df.index=range(0,len(df))
#Valeurs intermédiaires à partir de 2017

In [66]:
#On se concentre d'abord sur une année afin d'initialiser le filtre
df=df[(df.Date >'2016-01-01') & (df.Date <'2017-01-01')]
df.index=range(0,len(df))

### Temperatures data

In [67]:
df_temp=pd.read_csv(mypath + 'temp_2016.csv',sep=',')
df_temp=df_temp.drop(df_temp.columns[0],axis=1)
df_temp=df_temp.sort_values(by='date')

### Common parameters


In [68]:
nb_days = 15
nb_particles = 10**3
N_daytype = 9

## Gibbs Sampling For 3AM (03:00)

### Parameters required

In [69]:
#temperatures every day at 3am
three_am = "03:00"
temp_day_3=df_temp[df_temp.hour==three_am]
temp_day_3.head()
T_h_3=list(temp_day_3.t)

temperatures_3 = T_h_3[:nb_days] #france mean temp in jan 2016
daytypes_3 = list(df[df['Heure']==three_am]['daytype'][:nb_days])
consumptions_3 = list(df[df['Heure']==three_am]['Consommation (MW)'][:nb_days])

In [70]:
#k_day_3 = np.random.dirichlet(np.ones(N_daytype),1)
#kappa_3 = k_day_3[0].tolist()
kappa_3 = np.array([1/N_daytype]*N_daytype)

#u_heat_3 = np.random.normal(14,1)  #fixed parameter, invariant to time
u_heat_3 = 13

#sigma2_3 = 27000000
sigma2_3 = invgamma.rvs(a=10**(-2), scale=10**(2), size = 1)[0]

### Execute Gibbs sampling

In [72]:
x_init_median = 0
while (x_init_median > 10**5 or x_init_median < 10**4):
    GSM_3 = GibbsSamplingModel(temperatures_3,daytypes_3,consumptions_3,nb_days,nb_particles,sigma2_3,kappa_3,u_heat_3)

    start_time = time.time()
    GSM_3.gibbs_init()
    print("--- %s seconds ---" % (time.time() - start_time))

    x_init_median = np.median(GSM_3.x)

x_init_median

Gibbs sampling for particle 1/1000
Gibbs sampling for particle 100/1000
Gibbs sampling for particle 200/1000
Gibbs sampling for particle 300/1000
Gibbs sampling for particle 400/1000
Gibbs sampling for particle 500/1000
Gibbs sampling for particle 600/1000
Gibbs sampling for particle 700/1000
Gibbs sampling for particle 800/1000
Gibbs sampling for particle 900/1000
--- 3.9173202514648438 seconds ---
Gibbs sampling for particle 1/1000
Gibbs sampling for particle 100/1000
Gibbs sampling for particle 200/1000
Gibbs sampling for particle 300/1000
Gibbs sampling for particle 400/1000
Gibbs sampling for particle 500/1000
Gibbs sampling for particle 600/1000
Gibbs sampling for particle 700/1000
Gibbs sampling for particle 800/1000
Gibbs sampling for particle 900/1000
--- 3.83074688911438 seconds ---


24450.755762036366

### Save data

In [74]:
#Write initial parameters from Gibbs sampling in a file
#timestr = time.strftime("%Y%m%d-%H%M%S")

output_file_3 = mypath + 'parameters_init_3am.pkl'
parameters = {
    's_init':GSM_3.s[nb_days-1,],
    'g_heat_init':GSM_3.g_heat[nb_days-1,:],
    'sigma_s_init':np.sqrt(GSM_3.sigma_s_star_2[0,nb_particles-1]),
    'sigma_g_init':np.sqrt(GSM_3.sigma_g_star_2[0,nb_particles-1]),
    'x_init':GSM_3.x,
    'w_init':GSM_3.w,
    'sigma2': sigma2_3,
    'u_h':u_heat_3,
    'kappa': kappa_3
}
output_3 = open(output_file_3, 'wb')
pickle.dump(parameters, output_3)
output_3.close()

## Gibbs Sampling For 3PM (15:00)

### Parameters required

In [11]:
#temperatures every day at 3am
three_pm = "15:00"
temp_day_15=df_temp[df_temp.hour==three_pm]
temp_day_15.head()
T_h_15=list(temp_day_15.t)

temperatures_15 = T_h_15[:nb_days] #france mean temp in jan 2016
daytypes_15 = list(df[df['Heure']==three_pm]['daytype'][:nb_days])
consumptions_15 = list(df[df['Heure']==three_pm]['Consommation (MW)'][:nb_days])

In [15]:
#sigma2_15 = invgamma.rvs(a=10**(-2), scale=10**(2), size = 1)[0]
sigma2_15 = 27000000

#k_day_15 = np.random.dirichlet(np.ones(N_daytype),1)
#kappa_15 = k_day[0].tolist()
kappa_15 = np.array([1/N_daytype]*N_daytype)

#u_heat_15 = np.random.normal(14,1)  #fixed parameter, invariant to time
u_heat_15 = 13

### Execute Gibbs sampling

In [17]:
GSM_15 = GibbsSamplingModel(temperatures_15,daytypes_15,consumptions_15,nb_days,nb_particles,sigma2_15,kappa_15,u_heat_15)

In [18]:
start_time = time.time()
GSM_15.gibbs_init()
print("--- %s seconds ---" % (time.time() - start_time))

Gibbs sampling for particle 1/1000
Gibbs sampling for particle 100/1000
Gibbs sampling for particle 200/1000
Gibbs sampling for particle 300/1000
Gibbs sampling for particle 400/1000
Gibbs sampling for particle 500/1000
Gibbs sampling for particle 600/1000
Gibbs sampling for particle 700/1000
Gibbs sampling for particle 800/1000
Gibbs sampling for particle 900/1000
--- 4.150750160217285 seconds ---


In [20]:
np.median(BFM_15.x)

array([  6.27587845e+37,   6.95676015e+04,   6.53183041e+04,
         7.21392914e+04,   6.98536625e+04,   7.27918585e+04,
         6.79722981e+04,   7.24051805e+04,   6.87381481e+04,
         6.15175092e+04,   6.44169932e+04,   6.76497114e+04,
         7.02544870e+04,   7.58951525e+04,   5.88115695e+04,
         6.55264875e+04,   6.66222436e+04,   6.60591903e+04,
         6.74993977e+04,   5.41190487e+04,   6.42969987e+04,
         6.52033008e+04,   6.63350290e+04,   6.75999175e+04,
         7.20752981e+04,   6.23426257e+04,   6.46051750e+04,
         6.27691066e+04,   5.86754204e+04,   5.81531569e+04,
         6.46640137e+04,   7.36408485e+04,   5.84822266e+04,
         6.17458532e+04,   6.53182598e+04,   6.29600286e+04,
         6.99762416e+04,   7.29280087e+04,   6.76724497e+04,
         7.49992052e+04,   6.95162393e+04,   6.47369673e+04,
         6.41707950e+04,   6.42332185e+04,   6.55969958e+04,
         5.17568253e+04,   6.63900082e+04,   6.65490388e+04,
         6.13469962e+04,

### Save data

In [22]:
#Write initial parameters from Gibbs sampling in a file
#timestr = time.strftime("%Y%m%d-%H%M%S")

output_file_15 = mypath + 'parameters_init_3pm.pkl'
parameters = {
    's_init':GSM_15.s[nb_days-1,],
    'g_heat_init':GSM_15.g_heat[nb_days-1,:],
    'sigma_s_init':np.sqrt(GSM_15.sigma_s_star_2[0,nb_particles-1]),
    'sigma_g_init':np.sqrt(GSM_15.sigma_g_star_2[0,nb_particles-1]),
    'x_init':GSM_15.x,
    'w_init':GSM_15.w,
    'sigma2': sigma2_15,
    'u_h':u_heat_15,
    'kappa': kappa_15
}
output_15 = open(output_file_15, 'wb')
pickle.dump(parameters, output_15)
output_15.close()